In [54]:
import ast
import glob
import pandas as pd
from mastodon import Mastodon

In [ ]:
!pip install Mastodon.py

In [41]:
files = glob.glob('toots_from_sciencemastodon.com_*.tsv')

In [7]:
# timeline https://mastodonpy.readthedocs.io/en/stable/07_timelines.html

In [38]:
len(timeline)

90

In [45]:
users = []
fields = ['id', 'username', 'acct', 'created_at', 'following_count', 'followers_count']

for file in files:
    timeline = pd.read_csv(file, sep='\t')
    print(len(timeline))
    for row in timeline['account']:
        row = ast.literal_eval(row)
        user = []
        for field in fields:
            user.append(row[field])
        users.append(tuple(user))

users_timeline = pd.DataFrame(users)
users_timeline.to_csv('users_timeline_sciencemastodon.com.tsv', sep='\t')

90
15
14
13
12
11
10
9


In [52]:
mastodon_app = "acmbrito_app_python_secret"
client_id, client_secret, access_token, api_base_url = open(mastodon_app).read().split()
api1 = Mastodon(
    client_id=client_id,
    client_secret=client_secret,
    access_token=access_token,
    api_base_url=api_base_url
)

def get_userfollowers(user_id):
    followers = api1.account_followers(user_id)
    followers_id = []
    count = 0
    while followers:
        count += 1
        if count > 5:
            break
        for f in followers:
            followers_id.append(f['id'])
        followers = api1.fetch_next(followers)

    return followers_id

In [ ]:
def get_userid(handle):
    try:
        user_profile = api1.account_lookup(handle)
        return user_profile["id"]
    except Exception as error:
        print(error)
        return None
    
def get_users(user_id):
    followers = get_userfollowers(user_id)
    if followers:
        followers = set(followers)
        return followers
    else:
        return set()

for idx, row in users_timeline.iterrows():
    user_id = get_userid(row[2])
    test = get_users(user_id)
    